# SIEM Use Cases

- What IP address (and countries) are attacking the WAN endpoint?
 - [x] Visualise those against global map (grafana)
 - [ ] Block non AU countries

- Validate the firewall rules for VLAN isolation:
 - [ ] Flow from VLAN x to y is valid/not valid
 - [ ] Alert on not valid flows 

- Identify Unifi log and component anomalies
 - [ ] PAM activity ({job="ubnt-authpriv"} requires review)
 - [ ] Log interaction anomalies - new flows not seen before
- Detect loss of internet connectivity

In [5]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta

def get_logs_as_df(query, n_hours, step=100):
    dend = int(datetime.utcnow().timestamp())*1000000000
    dt = datetime.now() - timedelta(n_hours)
    dstart = int(dt.timestamp()*1000000000)

    url = "http://loki.loki:3100/loki/api/v1/query_range"
    headers = {'Content-type': 'application/json'}
    params = {
        'query': query,
        'direction': 'BACKWARD',
        'limit': 100,
        'start': dstart,
        'end': dend,
        'step': step
    }

    resp = requests.get(url, headers = headers, params = params)
    df = {}
    if (resp.status_code != 200):
        print("Failed to get data: " + resp.text)
        return {}
        
    json_obj = (json.loads(resp.text))

    # Refer https://github.com
    #   /BindiChen/machine-learning/blob/main/data-analysis/027-pandas-convert-json/pandas-convert-json.ipynb
    df = (pd.json_normalize(
        json_obj, 
        record_path = ['data','result','values'])
    )
    return df

In [6]:
# logs that contain external actors attempting to ingress via WAN
d = get_logs_as_df('{job="ubnt-geo"}', 3)
df = pd.DataFrame(d)
df.rename(columns = {0 : 'time', 1 : 'json'}, inplace = True)
df

,time,json
0,1651967125000000000,"{""geoplugin_request"":""45.95.147.24"",""geoplugin..."
1,1651967116000000000,"{""geoplugin_request"":""89.248.165.109"",""geoplug..."
2,1651967114000000000,"{""geoplugin_request"":""128.1.248.46"",""geoplugin..."
3,1651967108000000000,"{""geoplugin_request"":""89.248.165.109"",""geoplug..."
4,1651967106000000000,"{""geoplugin_request"":""157.245.212.8"",""geoplugi..."
...,...,...
95,1651966357000000000,"{""geoplugin_request"":""17.178.104.183"",""geoplug..."
96,1651966357000000000,"{""geoplugin_request"":""17.178.104.183"",""geoplug..."
97,1651966355000000000,"{""geoplugin_request"":""89.248.165.109"",""geoplug..."
98,1651966353000000000,"{""geoplugin_request"":""17.178.104.183"",""geoplug..."


In [7]:
from glom import glom
import re

# Find IPs in syslog msg: SRC and DEST. 
# We assume SRC is the first regex group (0).
IP4_REGEX = r'(?:[0-9]{1,3}\.){3}[0-9]{1,3}'
df1 = df[1].apply(lambda row: 
    re.search( 
        IP4_REGEX,
        glom(json.loads(row), 'geoplugin_request')
    ).group(0)
)
df1


KeyError: 1

In [3]:
# logs that have been dropped across VLANs
d = get_logs_as_df('{job="ubnt-kern"} |~"AN_OUT-.*-D"', 24, 3600)
df = pd.DataFrame(d)
df.rename(columns = {0 : 'time', 1 : 'json'}, inplace = True)
df
#df1 = pd.json_normalize(df, record_path=['json'],
#df1

,time,json
0,1651317976000000000,"{""facility"":""kern"",""facilityCode"":0,""severity""..."
1,1651317976000000000,"{""facility"":""kern"",""facilityCode"":0,""severity""..."
2,1651317976000000000,"{""facility"":""kern"",""facilityCode"":0,""severity""..."
3,1651317976000000000,"{""facility"":""kern"",""facilityCode"":0,""severity""..."
4,1651317975000000000,"{""facility"":""kern"",""facilityCode"":0,""severity""..."
...,...,...
95,1651317970000000000,"{""facility"":""kern"",""facilityCode"":0,""severity""..."
96,1651317970000000000,"{""facility"":""kern"",""facilityCode"":0,""severity""..."
97,1651317970000000000,"{""facility"":""kern"",""facilityCode"":0,""severity""..."
98,1651317970000000000,"{""facility"":""kern"",""facilityCode"":0,""severity""..."
